# Premier pas 

Ce notebook te permettra de faire tes premier pas en Python avec le robot Reachy 


**Note:** Un seul notebook à la fois peut faire marcher le robot, assure toi bien que personne n'est connecté sur le robot en même temps 

In [ ]:
import time
import cv2 as cv
import numpy as np
import time


from matplotlib import pyplot as plt

## Ce connecter au différentes pièces du robot

In [ ]:
from reachy import Reachy, parts 

reachy = Reachy(
    right_arm=parts.RightArm(io='/dev/ttyUSB*', hand='force_gripper'),
    ead=parts.Head(io='/dev/ttyUSB*'),
)

Regarder si tous les moteurs du bras sont présents :

In [ ]:
for m in reachy.right_arm.motors:
    print(f'{m.name} - pos:{m.present_position}')

Regarder si tous les 3 disque de la tête sont présent : 

In [ ]:
for d in reachy.head.neck.disks:
    print(d)

Tu peux bouger le bras et la tete et voir si les positions changent.

## Compliant ? 

Bras et tête compliant : 

In [ ]:
for m in reachy.right_arm.motors:
    m.compliant = True
    
reachy.head.compliant = True

Bras et tête non compliant : 

In [ ]:
for m in reachy.right_arm.motors:
    m.compliant = False

reachy.head.compliant = False

## Bouger le bras

#### Coude a 90°

In [ ]:
for m in reachy.right_arm.motors:
    m.compliant = False

In [ ]:
reachy.goto({
    'right_arm.shoulder_pitch': 0,
    'right_arm.shoulder_roll': 0,
    'right_arm.arm_yaw': 0,    
    'right_arm.elbow_pitch': -90,
    'right_arm.hand.forearm_yaw': 0,
    'right_arm.hand.wrist_pitch': 0,
    'right_arm.hand.wrist_roll': 0,
    'right_arm.hand.gripper': 0,
}, duration=3, wait=True)

for m in reachy.right_arm.motors:
    m.compliant = True

#### Enregistrer un mouvement et le reproduire

In [ ]:
from reachy.trajectory import TrajectoryRecorder, TrajectoryPlayer

In [ ]:
recorder = TrajectoryRecorder(reachy.right_arm.motors)

Le programme va enregistrer les déplacements du robot pendant 10 secondes. Fait attention a ce que le robot se trouve dans un endroit ou il peut bouger librement sans objets ou obstacles autour de lui. 

In [ ]:
recorder.start()
time.sleep(10)
recorder.stop()

Remttre le robot en mode non compliant

In [ ]:
for m in reachy.right_arm.motors:
    m.compliant = False

In [ ]:
player = TrajectoryPlayer(reachy, recorder.trajectories)
player.play(wait=True, fade_in_duration=3)

In [ ]:
for m in reachy.right_arm.motors:
    m.compliant = True

## Bouger la tête 

Aller à la position de base 

In [ ]:
reachy.head.compliant = False
reachy.head.look_at(1, 0, 0, duration=1, wait=True)

In [ ]:
reachy.head.compliant = True

Bouger la tête en rentrant la position voulu  

In [ ]:
reachy.head.compliant = False

**ATTENTION :** ne pas mettre la durée de déplacement trop rapide 

In [ ]:
reachy.head.neck.goto(thetas=(-10,-10,-10), duration=3, wait=True)

Faire bouger la tête avec une trajectoire aléatoire

In [ ]:
x = 0.5
y = (2 * np.random.rand() - 1) * 0.25
z = (2 * np.random.rand() - 1) * 0.25

duration = 1

reachy.head.look_at(x, y, z, duration=duration, wait=False)

real = []

t0 = time.time()
while time.time() - t0 < duration:
    real.append([d.rot_position for d in reachy.head.neck.disks])
    time.sleep(0.01)
    
plt.figure()
plt.plot(real)